#Gold Layer: Insights Extraction
 Where raw data is transformed into invaluable insights and strategic analyses. This stage marks the culmination of meticulous data processing, aggregation, and refinement, focusing on Lionel Messi's illustrious career in La Liga.

In [0]:
# Read Parquet file
df_spark_loaded = spark.read.format("parquet").load("/FileStore/tables/messi_silver_done.parquet")

In [0]:
# Create a temporary table
df_spark_loaded.createOrReplaceTempView("messi_temp_gold")

###Questions
#### How many goals did Messi have?
Lionel Messi scored 474 goals in La Liga, with 79% coming from Open Play, 12% from Penalty kicks, and 8% from Free Kicks.

In [0]:
%sql
SELECT 
    shot_type_name, 
    COUNT(*) as type_goals,
    ROUND((COUNT(*) * 100.0) / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM 
    messi_temp_gold
WHERE 
    shot_outcome_name = 'Goal'
    AND player_name = 'Lionel Andrés Messi Cuccittini' 
GROUP BY 
    shot_type_name
ORDER BY type_goals desc


shot_type_name,type_goals,percentage
Open Play,375,79.11
Penalty,60,12.66
Free Kick,39,8.23


#### Who are the top 5 players who provided the most assists to Messi?
Luis Suárez, Dani Alves, Pedro Ledesma, Andrés Iniesta and Sergio Busquets

In [0]:
%sql
SELECT 
    player_name, 
    COUNT(*) as assist
FROM 
    messi_temp_gold
WHERE 
    pass_goal_assist = TRUE
    AND pass_recipient_name = 'Lionel Andrés Messi Cuccittini' 
GROUP BY 
    player_name
ORDER BY assist desc LIMIT 5

player_name,assist
Luis Alberto Suárez Díaz,32
Daniel Alves da Silva,27
Pedro Eliezer Rodríguez Ledesma,18
Andrés Iniesta Luján,18
Sergio Busquets i Burgos,17


#### Who are the top 5 players who received the most assists from Messi?
Luis Suárez, Pedro Ledesma, Samuel Eto'o, Neymar Jr and David Villa

In [0]:
%sql
SELECT 
    pass_recipient_name, 
    COUNT(*) as assist
FROM 
    messi_temp_gold
WHERE 
    pass_goal_assist = TRUE
    AND player_name = 'Lionel Andrés Messi Cuccittini' 
GROUP BY 
    pass_recipient_name
ORDER BY assist desc LIMIT 5

pass_recipient_name,assist
Luis Alberto Suárez Díaz,30
Pedro Eliezer Rodríguez Ledesma,13
Samuel Eto''o Fils,12
Neymar da Silva Santos Junior,12
David Villa Sánchez,10


#### Which parts of the body did Messi use to score his goals?

The majority of Lionel Messi's goals in La Liga, 83%, were scored with his Left Foot, with the remaining 17% divided among Right Foot, Head, and other parts of his body. 


In [0]:
%sql
SELECT 
    shot_body_part_name, 
    COUNT(*) as body_part_goals,
    ROUND((COUNT(*) * 100.0) / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM 
    messi_temp_gold
WHERE 
    shot_outcome_name = 'Goal'
    AND player_name = 'Lionel Andrés Messi Cuccittini' 
GROUP BY 
    shot_body_part_name
ORDER BY body_part_goals desc

shot_body_part_name,body_part_goals,percentage
Left Foot,396,83.54
Right Foot,61,12.87
Head,16,3.38
Other,1,0.21


####In which position did he score the most goals?
The majority goals were scored while he was playing as a Right Wing (41%) and Center Forward (38%) in La Liga. 

In [0]:
%sql
SELECT 
    position_name, 
    COUNT(*) as position_player,
    ROUND((COUNT(*) * 100.0) / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM 
    messi_temp_gold
WHERE
    shot_outcome_name = 'Goal'
    AND player_name = 'Lionel Andrés Messi Cuccittini' 
GROUP BY 
    position_name
ORDER BY position_player desc

position_name,position_player,percentage
Right Wing,198,41.77
Center Forward,184,38.82
Right Center Forward,44,9.28
Center Attacking Midfield,30,6.33
Right Center Midfield,5,1.05
Left Center Forward,5,1.05
Right Attacking Midfield,4,0.84
Left Wing,3,0.63
Left Center Midfield,1,0.21


#### What were the outcomes of his shots (off the post, off target, saved, goals)?

Below is the table with each result of his shots on goal, but I would like to highlight a possible Pareto principle, as 20% of his shots on goal resulted in goals.

In [0]:
%sql
SELECT 
    shot_outcome_name, 
    COUNT(*) as shots,
    ROUND((COUNT(*) * 100.0) / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM 
    messi_temp_gold
WHERE
    shot_outcome_name IS NOT NULL
    AND player_name = 'Lionel Andrés Messi Cuccittini' 
GROUP BY 
    shot_outcome_name
ORDER BY shots desc

shot_outcome_name,shots,percentage
Saved,644,27.33
Blocked,571,24.24
Off T,541,22.96
Goal,474,20.12
Post,75,3.18
Wayward,30,1.27
Saved to Post,13,0.55
Saved Off Target,8,0.34


#### A fun fact: Lionel Messi completed 94 nutmegs during his career in La Liga.

In [0]:
%sql
SELECT 
    dribble_outcome_name,
    COUNT(*) as dribbles,
    ROUND((COUNT(*) * 100.0) / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM 
    messi_temp_gold
WHERE 
    dribble_nutmeg = TRUE
    AND player_name = 'Lionel Andrés Messi Cuccittini' 
GROUP BY 
    dribble_outcome_name 
ORDER BY dribbles desc 

dribble_outcome_name,dribbles,percentage
Complete,94,64.38
Incomplete,52,35.62


####Extra querys

In [0]:
%sql
SELECT 
    COUNT(*)
FROM 
    messi_temp_gold
WHERE
    shot_first_time = TRUE
    AND shot_outcome_name = 'Goal'
    AND player_name = 'Lionel Andrés Messi Cuccittini' 



count(1)
150


In [0]:
%sql
SELECT 
    shot_technique_name, 
    COUNT(*) as shots_technique,
    ROUND((COUNT(*) * 100.0) / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM 
    messi_temp_gold
WHERE
    shot_technique_name IS NOT NULL
    AND player_name = 'Lionel Andrés Messi Cuccittini' 
GROUP BY 
    shot_technique_name
ORDER BY shots_technique desc

shot_technique_name,shots_technique,percentage
Normal,2043,86.71
Half Volley,144,6.11
Volley,82,3.48
Lob,76,3.23
Diving Header,5,0.21
Overhead Kick,5,0.21
Backheel,1,0.04


In [0]:
%sql
SELECT 
    shot_technique_name, 
    COUNT(*) as shots_technique_on_goal,
    ROUND((COUNT(*) * 100.0) / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM 
    messi_temp_gold
WHERE
    shot_outcome_name = 'Goal'
    AND player_name = 'Lionel Andrés Messi Cuccittini' 
GROUP BY 
    shot_technique_name
ORDER BY shots_technique_on_goal desc

shot_technique_name,shots_technique_on_goal,percentage
Normal,393,82.91
Lob,32,6.75
Half Volley,27,5.70
Volley,19,4.01
Diving Header,3,0.63
